# Project Prototype: Agentic RAG System with Nursing Handbooks and Transes as Knowledge Base

The goal of this project is to create an **Agentic RAG-based system** that helps nursing students retrieve relevant information from nursing handbooks and their personal study notes. This system will augment the responses with context from the personal notes, making it more personalized and adaptive to the user's learning.

## Flow Overview

The project involves two primary stages:
1. **Prepopulating the Vector DB** (embedding the nursing handbooks and personal notes into a database)
2. **RAG Modeling** (retrieving relevant information and augmenting responses using both the handbooks and personal notes)

Additionally, there will be an **Agentic Layer** that intelligently routes queries to the appropriate source (nursing handbooks or personal study notes).

### Technologies to use

- **Docling**: For parsing the nursing handbooks and personal notes
- **OpenAI Embedding Model (large)**: For embedding both nursing handbooks and personal study notes.
- **ChromaDB**: For storing and querying the embeddings.
- **Deepseek LLM**: For augmenting responses based on the retrieved content.
- **Pydantic AI**: For AI agent intelligently routing queries between the nursing handbooks and personal notes. 
- **Pydantic**: For type safety and data validation.
- **FastAPI**: For building the API.
- **Docker**: For containerization and deployment.
- **Pydantic Graphs**: For workflow pipelines

## Initialization

In [70]:
import os
import joblib
import threading
import asyncio
import chromadb
import nanoid
import json
from pathlib import Path
import concurrent.futures
from rich import print
from docling.document_converter import DocumentConverter
from abc import ABC, abstractmethod
import chromadb.utils.embedding_functions as embedding_functions
from docling_core.transforms.chunker.hierarchical_chunker import DocChunk
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
from docling.chunking import HybridChunker

from dotenv import load_dotenv

load_dotenv()
knowledge_base_raw_path = './data/knowledge-base/raw/'
knowledge_base_pickled_path = './data/knowledge-base/pickled/'

### Stage 1: Prepopulate the Vector DB

In this stage, we process nursing handbooks and personal study notes, embedding them into a vector database for efficient retrieval during RAG modeling.

#### Steps:

1. **Upload Nursing Handbooks**:
   - Upload nursing handbooks in popular formats like PDF or DOCX.
   - These documents may include textbooks on topics like **medical-surgical nursing, pharmacology, pediatric nursing**, and other specialized nursing areas.

2. **Parsing the documents and retaining the page metadata**:
   - Use **Docling** to convert nursing handbooks and personal notes docling document.
   - The conversion ensures that the pdf metadata are preserved, making it easier for the AI to process.

3. **Generate Embeddings**:
   - Use **OpenAI's large embedding model** to convert the nursing handbooks and personal notes into embeddings. These embeddings will capture the semantic meaning of each section, allowing for efficient similarity-based searches.
   - Both the nursing handbooks and the personal study notes will be embedded into the vector database.

4. **Save to Vector DB (ChromaDB)**:
   - Store the generated embeddings in **ChromaDB** for fast retrieval during RAG modeling.
   - The vector database will allow the system to quickly access the most relevant information when a query is made.

**Note**: Converting and embedding long documents like nursing handbooks may take some time (e.g., **15-30 minutes** per document).



In [33]:
# List the raw files to convert
os.listdir(knowledge_base_raw_path)

['Handbook of Clinical Nursing_ Medical-Surgical Nursing -- Joyce Fitzpatrick -- 2018 -- Springer Publishing Company -- 9780826130785 -- 26f2533f396508e653d45e0e76aadc53 -- Anna’s Archive.pdf',
 'NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf']

In [34]:
document_converter = DocumentConverter()

knowledge_base_raw_files = os.listdir(knowledge_base_raw_path)
knowledge_base_pickled_files = os.listdir(knowledge_base_pickled_path)

for file in knowledge_base_raw_files:
	file_path = os.path.join(knowledge_base_raw_path, file)
	file_name = os.path.splitext(file)[0]

	# Check if not file 
	if not os.path.isfile(file_path):
		continue

	# Check if file is already pickled
	if f"{file_name}.docling" in knowledge_base_pickled_files:
		print(f"File `{file[:97]}`... is already pickled.")
		continue

	# Pickling the document to retain the metadata (there is currently no way to get the paging metadata from exported md)
	print(f"Pickling file: `{file[:97]}`...")
	conversion_result = document_converter.convert(file_path)
	
	joblib.dump(
		conversion_result.document, 
		os.path.join(knowledge_base_pickled_path, f"{file_name}.docling"),
		compress=3
	)

File `Handbook of Clinical Nursing_ Medical-Surgical Nursing -- Joyce Fitzpatrick -- 2018 -- Springer P`... is 
already pickled.

File `NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf`... is already 
pickled.

#### Create the collections

In [35]:
chroma_client = await chromadb.AsyncHttpClient(host="localhost", port="8001")

Using the OpenAI text-embedding-3-small model

In [36]:
embed_fn_openai = embedding_functions.OpenAIEmbeddingFunction(
	api_key=os.getenv("OPENAI_API_KEY"),
	model_name=os.getenv("OPENAI_EMBEDDING_MODEL"),
)

Creating the collections

In [37]:
collection_base_documents = await chroma_client.get_or_create_collection(name="base_documents", embedding_function=embed_fn_openai)

Chunking the document the NRG 304 document as test

In [38]:
document = joblib.load(os.path.join(
	knowledge_base_pickled_path, 
	"NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.docling"
))

### Stage 2: RAG Modeling

Once the vector database is populated, the system will retrieve relevant information from both nursing handbooks and personal study notes. The retrieval will be augmented using an LLM (Large Language Model) for more accurate and contextually rich responses.

#### Steps:

1. **Retrieve Relevant Information**:
   - When a query is input by the user (e.g., "What are the symptoms of diabetes?"), the system will retrieve the most relevant content from the vector database.
   - ChromaDB will find the sections of the nursing handbooks or personal study notes that are most similar to the query.

2. **Augment Response Using Deepseek LLM**:
   - The retrieved sections will be passed through the **Deepseek LLM**, which will generate a response based on the content of the handbooks and notes.
   - The LLM will combine the relevant information and format it into a coherent, accurate response tailored to the question.

In [39]:
chunker = HybridChunker(max_tokens=100)
chunk_iter = chunker.chunk(dl_doc=document)

for i, chunk in enumerate(chunk_iter):
	print(f"=== {i} ===")
	print(chunk.meta.export_json_dict())
	enriched_text = chunker.serialize(chunk=chunk)
	print(enriched_text)
	break

=== 0 ===

{
    'schema_name': 'docling_core.transforms.chunker.DocMeta',
    'version': '1.0.0',
    'doc_items': [
        {
            'self_ref': '#/tables/0',
            'parent': {'$ref': '#/body'},
            'children': [],
            'content_layer': 'body',
            'label': 'table',
            'prov': [
                {
                    'page_no': 1,
                    'bbox': {
                        'l': 19.69459342956543,
                        't': 521.74951171875,
                        'r': 280.4553527832031,
                        'b': 293.35418701171875,
                        'coord_origin': 'BOTTOMLEFT'
                    },
                    'charspan': [0, 0]
                }
            ]
        }
    ],
    'headings': ['BRIEF REVIEW OF THE ANATOMY OF GIT'],
    'origin': {
        'mimetype': 'application/pdf',
        'binary_hash': 7347013283832062728,
        'filename': 'NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf'
    }
}

BRIEF REVIEW OF THE ANATOMY OF GIT
Oral Cavity and Pharynx  ●  Injury/ loss or decay of teeth  ●  Atrophy of taste buds  ●  ↓ Saliva production  ●  
Reduced ptyalin and amylase in saliva, Age-Related Changes in the Gastrointestinal System.Implications = Difficulty
chewing and  swallowing. ●  ↓ Motility and emptying  ●  Weakened gag reflex  ●  ↓ Resting pressure of lower 
esophageal  sphincter

In [40]:
class Metadata(BaseModel):
	user_id: str
	filename: str
	heading: str
	page_no: int

class Records(BaseModel):
	documents: list[str] = []
	metadatas: list[Metadata] = []
	ids: list[str] = []

In [41]:
chunker = HybridChunker(max_tokens=1000)
chunk_iter = chunker.chunk(dl_doc=document)
document_chunks = list(chunk_iter)

records = Records()

# Lock for thread-safe appends
lock = threading.Lock()

def extract_and_store(chunk: DocChunk):
	enriched_text = chunker.serialize(chunk=chunk)
	chunk_id = nanoid.generate()
	doc_chunk_metadata = chunk.meta.export_json_dict()
	metadata = Metadata(
		user_id="jiya",
		filename=doc_chunk_metadata['origin']['filename'],
		heading=doc_chunk_metadata['headings'][0],
		page_no=doc_chunk_metadata['doc_items'][0]['prov'][0]['page_no']
	)

	with lock:
		records.documents.append(enriched_text)
		records.metadatas.append(metadata.model_dump())
		records.ids.append(chunk_id)

with concurrent.futures.ThreadPoolExecutor() as executor:
	executor.map(extract_and_store, document_chunks)

In [42]:
await collection_base_documents.add(
	documents=records.documents,
	metadatas=records.metadatas,
	ids=records.ids
)

In [104]:
query_result = await collection_base_documents.query(
    query_texts=["upper gastrointestinal tract study nursing interventions"],
    n_results=3,
    where={"user_id": "jiya"},
)

In [105]:
print(query_result)

{
    'ids': [['9tkIlw-GCYLqPiru48knp', 'pmU-Igw167TooAnmgGv5P', 'PFC71O1G4tWccO6aBOeKq']],
    'distances': [[0.8048333758256015, 0.822055825525987, 0.8433847393001588]],
    'embeddings': None,
    'metadatas': [
        [
            {
                'filename': 'NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL 
FUNCTION.pdf',
                'heading': 'Nursing Interventions :',
                'page_no': 2,
                'user_id': 'jiya'
            },
            {
                'filename': 'NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL 
FUNCTION.pdf',
                'heading': 'NURSING INTERVENTIONS',
                'page_no': 4,
                'user_id': 'jiya'
            },
            {
                'filename': 'NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL 
FUNCTION.pdf',
                'heading': '● Nursing Interventions',
                'page_no': 2,
                'user_id': 'jiya'
            }
        ]
    ],
    'documents': [
        [
            "Nursing Interventions :\n1. Clear  liquid  diet,  with  nothing  by  mouth  (NPO)  from midnight the 
night before the study.\n2. Patient is advised to not smoke or chew gum during the NPO period because these can 
increase gastric secretions and salivation.\n3. Polyethylene glycol (PEG)-based solutions are considered the most 
effective bowel cleansing preparatory agent.\n4. Oral  medications  are  withheld  on  the  morning  of  the study 
and  resumed  that  evening,  but  each  patient's medication regimen should be evaluated on an individual 
basis.\n5. When a patient with insulin dependent diabetes is NPO, their insulin requirements will need  to be  
adjusted accordingly.\n6. Instruct pt to increase OFI after the procedure  to facilitate evacuation of stool and 
barium.\n1. Low-residue diet 1 to 2 days before the test, a clear liquid diet and  a laxative  the  evening  
before,  NPO  after midnight,  and  cleansing  enemas  until  returns  are  clear the following morning.\n2. Enemas
are contraindicated for pts with active\n2",
            'NURSING INTERVENTIONS\n1. The patient is NPO for 8 to 12 hours before the procedure.\n2. Any 
medications that affect gastric secretions are withheld for 24 to 48 hours before the test.\n3. Smoking is not 
allowed on the morning of the test.\n4. Place pt in a semi-reclining position.\n5. Inform pt that a small 
nasogastric tube with a catheter tip marked at various  points  is inserted  through  the  nose.  The  entire  
stomach contents are aspirated by gentle suction into a syringe, and gastric samples are collected every 15 minutes
for the next hour.',
            '● Nursing Interventions\n:\n1. Instruct pt.  to  fast  for  8-12  hours  before  ultrasound 
testing.\n2. If  gallbladder  studies  are  being  3275  performed, the patient should eat a fat-free meal the 
evening before the test.\n3. If  barium studies are to be performed, they should be scheduled after 
ultrasonography.\n4. Patients who receive moderate sedation are observed for  about  1  hour  to  assess  for level
of consciousness, orientation, and ability to ambulate.\n5. Patients  treated  on  an  outpatient  basis  are  
given instructions regarding diet, activity, and how to monitor for complications.'
        ]
    ],
    'uris': None,
    'data': None,
    'included': ['distances', 'documents', 'metadatas']
}

In [106]:
documents = query_result["documents"][0]
metadatas = query_result["metadatas"][0]

for doc, metadata in zip(documents, metadatas):
	print(doc)
	print(f"Source: {metadata['filename']}, Page: {metadata['page_no']}")

Nursing Interventions :
1. Clear  liquid  diet,  with  nothing  by  mouth  (NPO)  from midnight the night before the study.
2. Patient is advised to not smoke or chew gum during the NPO period because these can increase gastric secretions 
and salivation.
3. Polyethylene glycol (PEG)-based solutions are considered the most effective bowel cleansing preparatory agent.
4. Oral  medications  are  withheld  on  the  morning  of  the study  and  resumed  that  evening,  but  each  
patient's medication regimen should be evaluated on an individual basis.
5. When a patient with insulin dependent diabetes is NPO, their insulin requirements will need  to be  adjusted 
accordingly.
6. Instruct pt to increase OFI after the procedure  to facilitate evacuation of stool and barium.
1. Low-residue diet 1 to 2 days before the test, a clear liquid diet and  a laxative  the  evening  before,  NPO  
after midnight,  and  cleansing  enemas  until  returns  are  clear the following morning.
2. Enemas are contraindicated for pts with active
2

Source: NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf, Page: 2

NURSING INTERVENTIONS
1. The patient is NPO for 8 to 12 hours before the procedure.
2. Any medications that affect gastric secretions are withheld for 24 to 48 hours before the test.
3. Smoking is not allowed on the morning of the test.
4. Place pt in a semi-reclining position.
5. Inform pt that a small nasogastric tube with a catheter tip marked at various  points  is inserted  through  the
nose.  The  entire  stomach contents are aspirated by gentle suction into a syringe, and gastric samples are 
collected every 15 minutes for the next hour.

Source: NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf, Page: 4

● Nursing Interventions
:
1. Instruct pt.  to  fast  for  8-12  hours  before  ultrasound testing.
2. If  gallbladder  studies  are  being  3275  performed, the patient should eat a fat-free meal the evening before
the test.
3. If  barium studies are to be performed, they should be scheduled after ultrasonography.
4. Patients who receive moderate sedation are observed for  about  1  hour  to  assess  for level of consciousness,
orientation, and ability to ambulate.
5. Patients  treated  on  an  outpatient  basis  are  given instructions regarding diet, activity, and how to 
monitor for complications.

Source: NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf, Page: 2

#### Response augmenting with deepseek r1

In [107]:
class Reference(BaseModel):
	content: str
	page_number: int
	filename: str
	heading: str

In [108]:
query_result['documents']

[["Nursing Interventions :\n1. Clear  liquid  diet,  with  nothing  by  mouth  (NPO)  from midnight the night before the study.\n2. Patient is advised to not smoke or chew gum during the NPO period because these can increase gastric secretions and salivation.\n3. Polyethylene glycol (PEG)-based solutions are considered the most effective bowel cleansing preparatory agent.\n4. Oral  medications  are  withheld  on  the  morning  of  the study  and  resumed  that  evening,  but  each  patient's medication regimen should be evaluated on an individual basis.\n5. When a patient with insulin dependent diabetes is NPO, their insulin requirements will need  to be  adjusted accordingly.\n6. Instruct pt to increase OFI after the procedure  to facilitate evacuation of stool and barium.\n1. Low-residue diet 1 to 2 days before the test, a clear liquid diet and  a laxative  the  evening  before,  NPO  after midnight,  and  cleansing  enemas  until  returns  are  clear the following morning.\n2. Enema

In [109]:
references: list[Reference] = []
for metadata, document in zip(query_result["metadatas"][0], query_result["documents"][0]):
	references.append(
		Reference(
			content=document,
			page_number=metadata['page_no'],
			filename=metadata['filename'],
			heading=metadata['heading']
		)
	)

In [113]:
class AgentDependencies(BaseModel):
	references: list[Reference]

model = OpenAIModel(
	'google/gemini-2.0-flash-thinking-exp:free',
	base_url=os.getenv("OPEN_ROUTER_BASE_URL"),
	api_key=os.getenv("OPEN_ROUTER_API_KEY"),
)

agent = Agent(
	model=model,
	deps_type=AgentDependencies,
	result_type=str,
	system_prompt="""
		# Role
		You are a helpful medical assistant. 
		
		# Task
		You are tasked to answer a question based on a set of documents that you have been provided with and then explain based on what your know about the matter. 
		Please provide a detailed and accurate answer to the question.
		Always include the references.

	""",
)


@agent.system_prompt
async def add_references(
	ctx: RunContext[AgentDependencies],
) -> str:
	references = ctx.deps.references

	return f"""
		# References
		{'\n'.join([json.dumps(references[0].model_dump()) for reference in references])}

	"""

In [114]:
result = await agent.run(
	"can you explain me how the nursing interventions for upper gastrointestinal tract study are done",
	deps = AgentDependencies(
		references = references
	)
)

In [115]:
print(result.data)

Upper gastrointestinal tract studies require specific nursing interventions to ensure the procedure's success and 
patient safety.  Based on the provided documents, here's how nursing interventions are performed:

**Pre-Procedure Interventions:**

1.  **Dietary Restrictions:** Patients are typically placed on a **clear liquid diet** starting the day before the 
study.  They must also be **NPO (nothing by mouth)** from midnight the night before. This restriction is crucial to
ensure the stomach is empty for optimal visualization during the study. (Reference:  "Nursing Interventions :", 
"page_number": 2, "filename": "NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL 
FUNCTION.pdf", "heading": "Nursing Interventions :", intervention 1)

2.  **Avoid Stimulants:** Patients should be advised **not to smoke or chew gum** during the NPO period. These 
activities can increase gastric secretions and salivation, potentially interfering with the study. (Reference: 
"Nursing Interventions :", "page_number": 2, "filename": "NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE
AND GASTROINTESTINAL FUNCTION.pdf", "heading": "Nursing Interventions :", intervention 2)

3.  **Bowel Preparation (General):** Although the documents mention bowel cleansing, and specifically Polyethylene 
glycol (PEG)-based solutions as effective, this is generally more focused on lower GI studies.  However, some level
of bowel preparation may still be advised depending on the specific upper GI study being performed. In some cases, 
a **low-residue diet** for 1-2 days before, followed by a clear liquid diet and potentially a **laxative** the 
evening before *might* be suggested, although not explicitly stated for *upper* GI studies in these documents. 
(Reference: "Nursing Interventions :", "page_number": 2, "filename": "NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF 
THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf", "heading": "Nursing Interventions :", intervention 3 and the 
numbered list following intervention 6).  **Cleansing enemas** are mentioned but are generally for lower bowel 
preparation and are **contraindicated for patients with active conditions** (Reference: "Nursing Interventions :", 
"page_number": 2, "filename": "NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL 
FUNCTION.pdf", "heading": "Nursing Interventions :", numbered list point 2).

4.  **Medication Management:**  **Oral medications are typically withheld** on the morning of the study and resumed
that evening. However, it's emphasized that **each patient's medication regimen should be evaluated individually.**
This is especially important for patients with conditions like **insulin-dependent diabetes**.  If a diabetic 
patient is NPO, their **insulin requirements will need to be adjusted accordingly** to prevent hypoglycemia. 
(Reference: "Nursing Interventions :", "page_number": 2, "filename": "NRG 304 LEC_ WEEK 2_CHAPTER 1 ASSESSMENT OF 
THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf", "heading": "Nursing Interventions :", interventions 4 and 5)

**Post-Procedure Interventions:**

1.  **Increase Oral Fluid Intake (OFI):** After the procedure, patients are instructed to **increase oral fluid 
intake (OFI)**. This is to facilitate the evacuation of stool and any contrast medium, like barium, that might have
been used during the study (although barium is more common in lower GI series; for upper GI, water-soluble contrast
might be used). (Reference: "Nursing Interventions :", "page_number": 2, "filename": "NRG 304 LEC_ WEEK 2_CHAPTER 1
ASSESSMENT OF THE DIGESTIVE AND GASTROINTESTINAL FUNCTION.pdf", "heading": "Nursing Interventions :", intervention 
6)


**Explanation based on General Knowledge:**

Upper gastrointestinal studies, such as esophagogastroduodenoscopy (EGD) or barium swallow, require the upper 
digestive tract to be clear for accurate visualization and diagnosis.

*   **NPO Status:**  An empty stomach is essential to p

### Stage 2: Agentic RAG System

The **Agentic Layer** adds an intelligent routing mechanism to the RAG system. Instead of simply retrieving data from a static source, the system will decide where to get the most relevant information based on the context of the query and whether the query pertains to the nursing handbooks or her personal study notes.

#### Steps:

0. **Incorporate Personal Study Notes (Transes)**:
   - Personal study notes (transes) are documents or annotations written by the user, capturing key insights, case studies, or specific experiences.
   - These notes will be embedded in the same way as the nursing handbooks and stored in the same vector database.

1. **Agent Routes the Query**:
   - When a user (nursing student) asks a question, the **agent** will decide whether to retrieve data from the nursing handbooks or her personal study notes.
   - The agent will consider:
     - The **context** of the question (general knowledge vs. personalized experiences).
     - Whether her **personal study notes** are more relevant (e.g., if she’s already written notes on a particular case or condition).
     - If the query is general, the agent will prioritize retrieving information from the **nursing handbooks**.

2. **Retrieve from Both Sources**:
   - The system retrieves information from both the **nursing handbooks** and **personal study notes** based on the agent’s decision.
   - If the question pertains to something she has written down in her personal notes, the agent will prioritize retrieving those, combining them with general knowledge from the handbooks.

3. **Augment Response Using Deepseek LLM**:
   - The retrieved content from both sources is fed to the **Deepseek LLM**, which will merge the insights and generate a complete response.
   - The LLM will ensure the answer is not only accurate but also personalized based on the notes, making it more useful and contextually rich.
